# Planet: Understanding the Amazon from Space

An exploration into which augmentations gives the best result.

In [ ]:
from fastai.vision.all import *

In [ ]:
path = Path("../input/planets-dataset/planet/planet")
Path.BASE_PATH = path

train_df = pd.read_csv(path/'train_classes.csv')

In [ ]:
class NoValidation(Callback):
    def before_validate(self): raise CancelValidException()

class LabelSmoothingBCEWithLogitsLossFlat(BCEWithLogitsLossFlat):
    def __init__(self, eps:float=0.1, **kwargs):
        self.eps = eps
        super().__init__(eps=eps, **kwargs)
    
    def __call__(self, inp, targ, **kwargs):
        # https://www.kaggle.com/c/siim-isic-melanoma-classification/discussion/166833#929222
        targ_smooth = targ.float() * (1. - self.eps) + 0.5 * self.eps
        return super().__call__(inp, targ_smooth, **kwargs)
    
    def __repr__(self):
        return "FlattenedLoss of LabelSmoothingBCEWithLogits()"
        
def get_items(df):
    return df
    # Grab subset of items
    return df.sample(n=64*30, random_state=891237)

def get_x(df_row):
    img_name = df_row['image_name']
    img_path = path/'train-jpg'/f'{img_name}.jpg'
    return img_path

def get_y(df_row):
    return df_row['tags'].split(' ')

def get_dls(bs:int=64, img_size:int=256, batch_tfms:list=[], item_tfms:list=[]):
    datablock = DataBlock(
        get_items=get_items,
        get_x=get_x,
        get_y=get_y,
        blocks=(ImageBlock, MultiCategoryBlock),
        splitter=RandomSplitter(0.5, seed=7355233),
        item_tfms=item_tfms,
        batch_tfms=[
            *batch_tfms,
            Normalize.from_stats(*imagenet_stats)
        ]
    )
    return datablock.dataloaders(train_df, bs=bs)

def get_learner(dls, loss_func, model=resnet50, opt_func=Adam):
    cbs = [ShowGraphCallback()]
    metrics=[accuracy_multi, FBetaMulti(2.0, 0.2, average='samples')]
    return cnn_learner(dls, arch=model, cbs=cbs, loss_func=loss_func, metrics=metrics, opt_func=opt_func).to_fp16()

def create_learner(batch_tfms={"aug_transforms": { "flip_vert": True, "max_lighting": 0.1, "max_zoom": 1.05, "max_warp": 0. }}, bs:int=64, loss_func=LabelSmoothingBCEWithLogitsLossFlat(), opt_func=ranger, **kwargs):
    _batch_tfms = [tfms for key,value in batch_tfms.items() for tfms in globals()[key](**value)]
    dls = get_dls(batch_tfms=_batch_tfms, bs=bs)
    return get_learner(dls, loss_func=loss_func, opt_func=opt_func)

## Starting point

Let's try running with the same hyper parameters that worked well for resnet34.

In [ ]:
bs      = 256
lr_max  = 2e-1
mom_max = 0.95
mom_min = mom_max * 0.9 # 0.85
wd      = 5e-7
opt_func = partial(ranger, mom=mom_max, wd=wd)
learn = create_learner(bs=bs, wd=wd, opt_func=opt_func, moms=(mom_min, mom_max, mom_min))

learn.fit_one_cycle(1, lr_max=lr_max)

In [ ]:
learn.save("planet_001")

In [ ]:
bs      = 200
lr_max  = 2e-1
mom_max = 0.95
mom_min = mom_max * 0.9 # 0.85
wd      = 5e-7
opt_func = partial(ranger, mom=mom_max, wd=wd)
learn = create_learner(bs=bs, wd=wd, opt_func=opt_func, moms=(mom_min, mom_max, mom_min))

learn.lr_find()

Using the learning rate finder, we get a new suggested learning rate of `5e-2`. Let's test it and see if it can beat `0.906487`.

In [ ]:
bs      = 200
lr_max  = 5e-2
mom_max = 0.95
mom_min = mom_max * 0.9 # 0.85
wd      = 5e-7
opt_func = partial(ranger, mom=mom_max, wd=wd)
learn = create_learner(bs=bs, wd=wd, opt_func=opt_func, moms=(mom_min, mom_max, mom_min))

learn.fit_one_cycle(1, lr_max=lr_max)

`0.908423` is an improvement. Let's go with that.

In [ ]:
learn.save("planet_002")

In [ ]:
bs      = 200
lr_max  = 5e-2
mom_max = 0.95
mom_min = mom_max * 0.9 # 0.85
wd      = 5e-7
opt_func = partial(ranger, mom=mom_max, wd=wd)
learn = create_learner(bs=bs, wd=wd, opt_func=opt_func, moms=(mom_min, mom_max, mom_min))

learn.load("planet_002")
learn.unfreeze()

learn.lr_find()

The loss starts increasing at `3e-3`. 1/10th of that, `3e-4`, should be a good learning rate. 

In [ ]:
bs      = 200
lr_max  = 3e-4
mom_max = 0.95
mom_min = mom_max * 0.9 # 0.85
wd      = 5e-7
opt_func = partial(ranger, mom=mom_max, wd=wd)
learn = create_learner(bs=bs, wd=wd, opt_func=opt_func, moms=(mom_min, mom_max, mom_min))

learn.load("planet_002")
learn.unfreeze()

learn.fit_one_cycle(5, lr_max=lr_max)

`0.925709` is not really an improvement.

Let's try some other learning rates to see if they are any better.

In [ ]:
bs      = 200
lr_max  = 5e-4
mom_max = 0.95
mom_min = mom_max * 0.9 # 0.85
wd      = 5e-7
opt_func = partial(ranger, mom=mom_max, wd=wd)
learn = create_learner(bs=bs, wd=wd, opt_func=opt_func, moms=(mom_min, mom_max, mom_min))

learn.load("planet_002")
learn.unfreeze()

learn.fit_one_cycle(5, lr_max=lr_max)

In [ ]:
bs      = 200
lr_max  = 1e-3
mom_max = 0.95
mom_min = mom_max * 0.9 # 0.85
wd      = 5e-7
opt_func = partial(ranger, mom=mom_max, wd=wd)
learn = create_learner(bs=bs, wd=wd, opt_func=opt_func, moms=(mom_min, mom_max, mom_min))

learn.load("planet_002")
learn.unfreeze()

learn.fit_one_cycle(5, lr_max=lr_max)

In [ ]:
bs      = 200
lr_max  = 1e-4
mom_max = 0.95
mom_min = mom_max * 0.9 # 0.85
wd      = 5e-7
opt_func = partial(ranger, mom=mom_max, wd=wd)
learn = create_learner(bs=bs, wd=wd, opt_func=opt_func, moms=(mom_min, mom_max, mom_min))

learn.load("planet_002")
learn.unfreeze()

learn.fit_one_cycle(5, lr_max=lr_max)

In [ ]:
bs      = 200
lr_max  = 3e-4
mom_max = 0.95
mom_min = mom_max * 0.9 # 0.85
wd      = 5e-7
opt_func = partial(ranger, mom=mom_max, wd=wd)
learn = create_learner(bs=bs, wd=wd, opt_func=opt_func, moms=(mom_min, mom_max, mom_min))

learn.load("planet_002")
learn.unfreeze()

learn.fit_one_cycle(10, lr_max=lr_max)

In [ ]:
bs      = 200
lr_max  = 3e-4
mom_max = 0.95
mom_min = mom_max * 0.9 # 0.85
wd      = 5e-7
opt_func = partial(ranger, mom=mom_max, wd=wd)
learn = create_learner(bs=bs, wd=wd, opt_func=opt_func, moms=(mom_min, mom_max, mom_min))

learn.load("planet_002")
learn.unfreeze()

learn.fit_one_cycle(15, lr_max=lr_max)

## Submission

In [ ]:
file_path = Path('../input/planets-dataset/test-jpg-additional/test-jpg-additional')
test_path = Path('../input/planets-dataset/planet/planet/test-jpg')
submission_df = pd.read_csv(path/'sample_submission.csv')
testing_path = (submission_df['image_name'] + '.jpg').apply(lambda x: test_path/x if x.startswith('test') else file_path/x)

def prediction(filename='submission.csv', tta=False):
    tst_dl = learn.dls.test_dl(testing_path)
    if tta:
        predictions = learn.tta(dl = tst_dl)
    else:
        predictions = learn.get_preds(dl = tst_dl)
    predlist = [' '.join(learn.dls.vocab[i]) for i in (predictions[0] > 0.2)]

    df = submission_df
    df['tags'] = predlist

    df.to_csv(filename, index=False)
    return df

In [ ]:
prediction('submission.csv', tta=False)

In [ ]:
prediction('submission-tta.csv', tta=True)